# Archery Antalya Analysis

This notebook performs the analysis of the 2025 Antalya leg of the World Archery
world cup.

At this event new experimental rounds were used.

Colloquially referred to as the WA 660 rounds, these were shot on the same size faces
and at the same distances as other world cup legs, but featured two key differences:

- 60 arrows per round rather than the usual 72, and
- 11-zone scoring where the X-ring scores 11 points, rather than 10-zone scoring.

This repository takes a look at the results to see how these changes affected some
aspects of the competition.

In [ ]:
# Import required packages for analysis

import os

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from antalya_analysis import general_routines as gr
from antalya_analysis import ianseo_scrape as ianseo

## Set up files to store data and results

Define the variables:

- `mydata: str` - directory where any raw data you download from ianseo will be saved
- `myresults: str` - directory where any results will be saved

In [ ]:
mydata = "data/"
myresults = "results/"

Check these directories exist, and if not create them.

In [ ]:
datapath = f"{os.getcwd()}/{mydata}/"
resultspath = f"{os.getcwd()}/{myresults}/"
if not os.path.exists(datapath):
    os.makedirs(datapath)
if not os.path.exists(resultspath):
    os.makedirs(resultspath)

## Retrieve the data from ianseo

Datasets are defined as a dictionary with the key being a 'human-readable' identifier to each competition/event
and the corresponding item being a url linking to the base ianseo results page for that event.

Here we define two a single dataset with a single entry for Antalya 2025:

In [ ]:
dataset_name = "Antalya 2025"
dataset = {"Antalya 2025": "https://ianseo.net/TourData/2025/23008/"}

The following cell will retrieve the data for the requested events from ianseo and save combined results from each event as a `.csv` file in the directory specified above in the `mydata` variable.

> Warning: If this cell has already been run then it will _overwrite_ any previously saved with a new download according to the request.

In [ ]:
for id, url in dataset.items():
    print(f"Processing {id}...", end="")

    # Fetch Recurve and Compound Results for Men and Women and combine into one table
    RM = ianseo.get_cat(url, "IQRM.php", div="R", gen="M")
    RW = ianseo.get_cat(url, "IQRW.php", div="R", gen="W")
    CM = ianseo.get_cat(url, "IQCM.php", div="C", gen="M")
    CW = ianseo.get_cat(url, "IQCW.php", div="C", gen="W")
    full_results = pd.concat([RM, RW, CM, CW], ignore_index=True)

    # Save to file
    full_results.to_csv(f"{datapath}/{id.replace(' ', '_')}.csv")
    print("Done")
print("\nAll datasets downloaded and processed.")

## Process the data

Read in all of the data from each event in the dataset into a single dataframe.

Then process to to:

- Calculate score without 11-zone scoring
- Calculate rank without 11-zone scoring

Read in data from files and display to visualise what we have:

In [ ]:
df_all = gr.read_from_files(
    dataset.keys(),
    datapath=datapath,
    fname_fmt=".csv",
)

df_all

In [ ]:
# Calculate 10-zone score by subtracting number of 11s from total score
df_all['Score 10-zone'] = df_all['Score'] - df_all['11']

# Calculate rankings based on 'Score_10_zone' within each Division and Class, resolving ties by 11s (Xs)
df_all['Rank 10-zone'] = (
    df_all.groupby(['Division', 'Class'], group_keys=False)
    .apply(lambda x: x.sort_values(['Score 10-zone', '11'], ascending=[False, False])
          .rank(method='first', ascending=False)['Score 10-zone'], include_groups=False)
    .astype(int)
)

# Calculate the change in ranking
df_all['Rank Change'] = df_all['Category Rank'] - df_all['Rank 10-zone']

# Display the updated DataFrame
df_all

In [ ]:
# Get unique combinations of Class and Division
unique_combinations = df_all[['Class', 'Division']].drop_duplicates()

# Set up the subplots grid
nrows = len(unique_combinations)

In [ ]:
fig, axes = plt.subplots(nrows=nrows, figsize=(10, 6 * nrows), sharex=True, sharey=True)

# Generate a plot for each combination of Class and Division
for i, (class_, division) in enumerate(unique_combinations.values):
    ax = axes[i] if nrows > 1 else axes  # Handle single subplot case
    subset = df_all[(df_all['Class'] == class_) & (df_all['Division'] == division)]
    
    ax.scatter(subset['Category Rank'], subset['Rank 10-zone'], alpha=0.7, label=f"{division} - {class_}")
    ax.set_title(f"Division: {division}, Class: {class_}")
    ax.set_xlabel('Category Rank')
    ax.set_ylabel('Rank 10-zone')
    ax.legend()
    ax.grid(True)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=nrows, figsize=(10, 6 * nrows), sharex=True, sharey=True)

# Generate a plot for each combination of Class and Division
for i, (class_, division) in enumerate(unique_combinations.values):
    ax = axes[i] if nrows > 1 else axes  # Handle single subplot case
    subset = df_all[(df_all['Class'] == class_) & (df_all['Division'] == division)]
    
    ax.scatter(subset['Category Rank'], subset['Rank Change'], alpha=0.7, label=f"{division} - {class_}")
    ax.set_title(f"Division: {division}, Class: {class_}")
    ax.set_xlabel('Category Rank')
    ax.set_ylabel('Rank Change')
    ax.legend()
    ax.grid(True)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# Get unique combinations of Division and Class...
unique_combinations = df_all[['Division', 'Class']].drop_duplicates()

# Set up the subplots grid with shared x-axis
nrows = len(unique_combinations)
fig, axes = plt.subplots(nrows=nrows, figsize=(12, 6 * nrows), sharex=True)

# Generate a plot for each combination of Division and Class
for i, (division, class_) in enumerate(unique_combinations.values):
    ax_main = axes[i] if nrows > 1 else axes  # Handle single subplot case
    subset = df_all[(df_all['Division'] == division) & (df_all['Class'] == class_)]
    
    # Primary y-axis: Scatter plot for Category Rank
    ax_main.scatter(subset['Category Rank'], subset['Category Rank'], alpha=0.9, color='blue', s=50)
    
    # Scatter plot for Rank_10_zone with color based on Rank Change (green to red)
    norm = plt.Normalize(subset['Rank Change'].min(), subset['Rank Change'].max())
    colors = cm.RdYlGn(norm(subset['Rank Change']))
    ax_main.scatter(subset['Category Rank'], subset['Rank 10-zone'], alpha=0.7, c=colors, edgecolors='black')
    
    # Add arrows connecting the two points
    for _, row in subset.iterrows():
        ax_main.annotate(
            '', 
            xy=(row['Category Rank'], row['Rank 10-zone']), 
            xytext=(row['Category Rank'], row['Category Rank']), 
            arrowprops=dict(arrowstyle='->', color='gray', lw=1)
        )
    
    # Set axis labels and title for the main panel
    ax_main.set_title(f"Division: {division}, Class: {class_}", fontsize=14)
    ax_main.set_xlabel('Category Rank', fontsize=12)
    ax_main.set_ylabel('Rank (Category Rank and Rank 10-zone)', fontsize=12)
    ax_main.grid(True)
    ax_main.invert_yaxis()  # Invert the primary y-axis

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Get unique combinations of Division and Class...
unique_combinations = df_all[['Division', 'Class']].drop_duplicates()

# Set up the subplots grid with shared x-axis
nrows = len(unique_combinations)
fig, axes = plt.subplots(nrows=nrows, figsize=(12, 6 * nrows), sharex=True)

# Determine the symmetric y-axis limits
y_max = max(abs(df_all['Rank Change'].min()), abs(df_all['Rank Change'].max()))

# Generate a plot for each combination of Division and Class
for i, (division, class_) in enumerate(unique_combinations.values):
    ax = axes[i] if nrows > 1 else axes  # Handle single subplot case
    subset = df_all[(df_all['Division'] == division) & (df_all['Class'] == class_)]
    
    # Scatter plot of Rank Change against Score
    ax.scatter(subset['Score'], subset['Rank Change'], alpha=0.7, color='blue')
    
    # Set axis labels and title
    ax.set_title(f"Division: {division}, Class: {class_}", fontsize=14)
    ax.set_xlabel('Score', fontsize=12)
    ax.set_ylabel('Rank Change', fontsize=12)
    ax.grid(True, linestyle='--', alpha=0.7)
    
    # Set symmetric y-axis limits
    ax.set_ylim(-y_max, y_max)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# Overall statistics for Rank Change
overall_stats = df_all['Rank Change'].describe()
overall_stats['median'] = df_all['Rank Change'].median()

# Overall statistics for absolute Rank Change
overall_abs_stats = df_all['Rank Change'].abs().describe()
overall_abs_stats['median'] = df_all['Rank Change'].abs().median()

# Statistics for Rank Change grouped by Division
division_stats = df_all.groupby('Division')['Rank Change'].describe()
division_stats['median'] = df_all.groupby('Division')['Rank Change'].median()

# Statistics for absolute Rank Change grouped by Division
division_abs_stats = df_all.groupby('Division')['Rank Change'].apply(lambda x: x.abs().describe())
division_abs_stats['median'] = df_all.groupby('Division')['Rank Change'].apply(lambda x: x.abs().median())

# Display the results
print("Overall Rank Change Statistics:")
print(overall_stats)

print("\nOverall Absolute Rank Change Statistics:")
print(overall_abs_stats)

print("\nRank Change Statistics by Division:")
print(division_stats)

print("\nAbsolute Rank Change Statistics by Division:")
print(division_abs_stats)

In [ ]:
# Overall statistics for absolute Rank Change
overall_abs_stats = df_all['Rank Change'].abs().describe()
overall_abs_stats['median'] = df_all['Rank Change'].abs().median()

# Statistics for absolute Rank Change grouped by Division and Class
group_abs_stats = df_all.groupby(['Division', 'Class'])['Rank Change'].apply(lambda x: x.abs().describe())
group_abs_stats['median'] = df_all.groupby(['Division', 'Class'])['Rank Change'].apply(lambda x: x.abs()).median()

# Display the results
print("\nOverall Absolute Rank Change Statistics:")
print(overall_abs_stats)

print("\nAbsolute Rank Change Statistics by Division and Class:")
group_abs_stats

In [ ]:
group_median_abs = df_all.groupby(['Division', 'Class'])['Rank Change'].apply(lambda x: x.abs().median())

group_median_abs